In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F

import scipy
import scipy.signal
import scipy.io

import matplotlib.pyplot as plt
plt.style.use('dark_background')

from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import TransformedTargetRegressor
from joblib.parallel import Parallel, delayed


import dataloader as dl
import models as models
import train as train

import wandb as wandb

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: adrian-dendorfer (adrian_s_playground). Use `wandb login --relogin` to force relogin


In [8]:
h_params = {
    "num_epochs": 10, 
    "batch_size": 32, # 32 is too big for cuda; 16 to overfit it the most 
    "hidden_size1": 4096, # 512, 
    "hidden_size2": 128, 
    "lr": 0.002,  

    # "number_samples": 1 # number or "None" for all data.     >> is in dataloader
}

In [3]:
# # Get datasets (hier und nicht im dataloader file, um auf die h_params zugreifen zu können?)

# train_dataset = dl.EEGDataset(dl.data_raw_train, labels_raw_train) #, h_params["number_samples"]) #, number_samples=32) #!!! CHANGE AGAIN!! 
# val_dataset = dl.EEGDataset(data_raw_val, labels_raw_val) #, h_params["number_samples"]) #, number_samples=32) #!!! CHANGE AGAIN!! 
# test_dataset = dl.EEGDataset(data_raw_test, labels_raw_test) #, h_params["number_samples"]) #, number_samples=32) #!!! CHANGE AGAIN!! 



# Create Dataloaders
train_loader = DataLoader(dl.train_dataset, batch_size=h_params["batch_size"], shuffle=True, num_workers=4)
val_loader = DataLoader(dl.val_dataset, batch_size=h_params["batch_size"], shuffle=True, num_workers=4)
test_loader = DataLoader(dl.test_dataset, batch_size=h_params["batch_size"], shuffle=False, num_workers=4)

model = models.SimpleClassifierNN(61*100, 256, 32)

criterion = nn.BCELoss() 
optimizer = optim.AdamW(model.parameters(), lr=h_params["lr"])  


In [4]:
for i in enumerate(train_loader): 
    # print(i[0]) 
    print(i[1][0].shape)

torch.Size([1, 61, 100])


In [5]:
for i in enumerate(val_loader): 
    # print(i[0]) 
    print(i[1][0].shape)

torch.Size([1, 61, 100])


In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model.to(device)

cuda:0


SimpleClassifierNN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=6100, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=1, bias=True)
  (activation): ReLU()
  (final_activation): Sigmoid()
)

In [9]:
val_loader = train_loader

In [10]:
train.train_model(model, train_loader, val_loader, h_params, device)

Problem at: /u/home/dena/.conda/envs/mae2/lib/python3.11/site-packages/wandb/sdk/wandb_init.py 848 getcaller


CommError: Run initialization has timed out after 90.0 sec. 
Please refer to the documentation for additional information: https://docs.wandb.ai/guides/track/tracking-faq#initstarterror-error-communicating-with-wandb-process-

In [7]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_loader:
        X, y = data
        X,y = X.to(device), y.to(device)
        # calculate outputs by running images through the network
        outputs = model(X)
        # the class with the highest energy is what we choose as prediction
        # _, predicted = torch.max(outputs.data, 1)
        total += y.size(0)
        correct += (outputs == y).sum().item()

print(f'Accuracy of the network on the test set: {100 * correct // total} %')

Accuracy of the network on the test set: 0 %


In [ ]:
dataiter = iter(test_loader)
data_test = next(dataiter)
# data_test
test_X = data_test[0].to(device)
output = model(test_X)

plt.plot(output.to("cpu").detach().numpy())
plt.plot(data_test[1].to("cpu").detach().numpy())